# Konlpy Mecab 설치

In [ ]:
!pip install konlpy

     |████████████████████████████████| 19.4 MB 1.2 MB/s 
     |████████████████████████████████| 448 kB 41.6 MB/s 
     |████████████████████████████████| 86 kB 5.3 MB/s 
  Attempting uninstall: beautifulsoup4
    Found existing installation: beautifulsoup4 4.6.3
    Uninstalling beautifulsoup4-4.6.3:
      Successfully uninstalled beautifulsoup4-4.6.3


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!set -x \
&& pip install konlpy \
&& curl -s https://raw.githubusercontent.com/konlpy/konlpy/master/scripts/mecab.sh | bash -x

+ pip install konlpy
+ bash -x
+ curl -s https://raw.githubusercontent.com/konlpy/konlpy/master/scripts/mecab.sh
+ mecab_dicdir=/usr/local/lib/mecab/dic/mecab-ko-dic
+ set -e
++ uname
+ os=Linux
+ [[ ! Linux == \L\i\n\u\x ]]
+ hash sudo
+ sudo=sudo
+ python=python3
+ hash pyenv
+ at_user_site=
++ check_python_site_location_is_writable
++ python3 -
+ [[ 1 == \0 ]]
+ hash automake
+ echo 'Installing automake (A dependency for mecab-ko)'
Installing automake (A dependency for mecab-ko)
+ install_automake
+ '[' Linux == Linux ']'
++ grep -Ei 'debian|buntu|mint' /etc/lsb-release /etc/os-release
+ '[' '/etc/lsb-release:DISTRIB_ID=Ubuntu
/etc/lsb-release:DISTRIB_DESCRIPTION="Ubuntu 18.04.5 LTS"
/etc/os-release:NAME="Ubuntu"
/etc/os-release:ID=ubuntu
/etc/os-release:ID_LIKE=debian
/etc/os-release:PRETTY_NAME="Ubuntu 18.04.5 LTS"
/etc/os-release:HOME_URL="https://www.ubuntu.com/"
/etc/os-release:SUPPORT_URL="https://help.ubuntu.com/"
/etc/os-release:BUG_REPORT_URL="https://bugs.launchpad.net/ubu

In [ ]:
from konlpy.tag import Mecab
mecab = Mecab()

In [ ]:
mecab.pos('집에 가고 싶다.')

[('집', 'NNG'),
 ('에', 'JKB'),
 ('가', 'VV'),
 ('고', 'EC'),
 ('싶', 'VX'),
 ('다', 'EF'),
 ('.', 'SF')]

# 데이터 불러오기

In [ ]:
!apt-get -qq install -y graphviz && pip install pydot
import pydot

In [ ]:
!pip install pyLDAvis

     |████████████████████████████████| 1.7 MB 12.5 MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 11.3 MB 45.3 MB/s 
     |████████████████████████████████| 15.7 MB 59 kB/s 
  Created wheel for pyLDAvis: filename=pyLDAvis-3.3.1-py2.py3-none-any.whl size=136897 sha256=5cbda09b158cd7d88b2f4fd9006d7da9ec28f14e9734ed927e678cb651480710
  Stored in directory: /root/.cache/pip/wheels/c9/21/f6/17bcf2667e8a68532ba2fbf6d5c72fdf4c7f7d9abfa4852d2f
Successfully built pyLDAvis
  Attempting uninstall: numpy
    Found existing installation: numpy 1.19.5
    Uninstalling numpy-1.19.5:
      Successfully uninstalled numpy-1.19.5
  Attempting uninstall: pandas
    Found existing installation: pandas 1.1.5
    Uninstalling pandas-1.1.5:
      Successfully uninstalled pandas-1.1.5
ERROR: pip's dependency resolver does not currently take int

In [ ]:
#필요한 모듈 임포트하기
import pandas as pd
from pandas import DataFrame  as df
import numpy as np
from collections import Counter
from konlpy.tag import Okt, Mecab
okt = Okt()
mecab = Mecab()
%matplotlib inline
import matplotlib.pyplot as plt
import re 
import os
import gensim
from gensim import corpora, models
from gensim.models import CoherenceModel
from wordcloud import WordCloud
from datetime import datetime

In [ ]:
# 데이터 위치

path="/content/drive/MyDrive/2021 데컬코마니 북커톤/data/가사/골라낸 가사/골라낸 가사_전처리 후/IU/"
file_list = os.listdir(path)
files = [file for file in file_list if file.endswith('.txt')]
print(files)
print(len(files))

['가을아침.txt', 'Love Poem.txt', '밤편지.txt', '아이와 나의 바다.txt', '비밀의 화원.txt', '너의 의미.txt', '이름에게.txt', '마음.txt', '무릎.txt', '이런 엔딩.txt', '자장가.txt', '나의 옛날이야기.txt', '그대네요.txt', '시간의 바깥.txt', '에필로그.txt', 'Someday.txt', '나만 몰랐던 이야기.txt', '푸르던.txt', '미아.txt', '봄 안녕 봄.txt', '사랑이 지나가면.txt', '여자라서.txt', '비밀.txt', '마침표.txt', '싫은 날.txt', '첫 겨울이니까.txt', '안경.txt', '그 애 참 싫다.txt', '기차를 타고.txt', '여름밤의 꿈.txt', '느리게 하는 일.txt', '개여울.txt', '별을 찾는 아이.txt', '혼자 있는 방.txt', '그렇게 사랑은.txt', 'Last Fantasy.txt', '꽃.txt', '마음을 드려요.txt', '벽지 무늬.txt', '아침 눈물.txt', '사랑니.txt', '한낮의 꿈.txt']
42


In [ ]:
df=pd.DataFrame(columns=['filename', 'contents', 'nouns', 'keywords', 'topic'])
df

,filename,contents,nouns,keywords,topic


In [ ]:
all_contents=[]
for i,f in enumerate(files):
    with open (path+f) as file_path:
        contents=file_path.read()
        df=df.append({'filename':f, 'contents':contents}, ignore_index=True)
        #print(file_path)
        all_contents.append(contents)
        #print(all_contents[i:i+1])

In [ ]:
df

,filename,contents,nouns,keywords,topic
0,가을아침.txt,이른 아침 작은 새들 노랫소리 들려오면 언제나 그랬듯 아쉽게 잠을 깬다. 창문 하나...,NaN,NaN,NaN
1,Love Poem.txt,이른 아침 작은 새들 노랫소리 들려오면 언제나 그랬듯 아쉽게 잠을 깬다. 창문 하나...,NaN,NaN,NaN
2,밤편지.txt,이 밤 그날의 반딧불을 당신의 창 가까이 보낼게요. 음 사랑한다는 말이에요. 나 우...,NaN,NaN,NaN
3,아이와 나의 바다.txt,그러나 시간이 지나도 아물지 않는 일들이 있지 내가 날 온전히 사랑하지 못해서 맘이...,NaN,NaN,NaN
4,비밀의 화원.txt,바람을 타고 날아오르는 새들은 걱정 없이 아름다운 태양 속으로 음표가 되어 나네. ...,NaN,NaN,NaN
5,너의 의미.txt,너의 그 한 마디 말도 그 웃음도 나에겐 커다란 의미 너의 그 작은 눈빛도 쓸쓸한 ...,NaN,NaN,NaN
6,이름에게.txt,꿈에서도 그리운 목소리는 이름 불러도 대답을 하지 않아. 글썽이는 그 메아리만 돌아...,NaN,NaN,NaN
7,마음.txt,툭 웃음이 터지면 그건 너 쿵 내려앉으면은 그건 너 축 머금고 있다면 그건 너 둥 ...,NaN,NaN,NaN
8,무릎.txt,모두 잠드는 밤에 혼자 우두커니 앉아. 다 지나버린 오늘을 보내지 못하고서 깨어있어...,NaN,NaN,NaN
9,이런 엔딩.txt,안녕 오랜만이야. 물음표 없이 참 너다운 목소리 정해진 규칙처럼 추운 문가에 늘 똑...,NaN,NaN,NaN


In [ ]:
all_contents

['이른 아침 작은 새들 노랫소리 들려오면 언제나 그랬듯 아쉽게 잠을 깬다. 창문 하나 햇살 가득 눈부시게 비쳐오고 서늘한 냉기에 재채기할까 말까 음 눈 비비며 빼꼼히 창밖을 내다보니 삼삼오오 아이들은 재잘대며 학교 가고 산책 갔다 오시는 아버지의 양손에는 효과를 알 수 없는 약수가 하나 가득 음 딸각딸각 아침 짓는 어머니의 분주함과 엉금엉금 냉수 찾는 그 아들의 게으름이 상큼하고 깨끗한 아침의 향기와 구수하게 밥 뜸드는 냄새가 어우러진 가을 아침 내겐 정말 커다란 기쁨이야. 가을 아침 내겐 정말 커다란 행복이야. 응석만 부렸던 내겐 파란 하늘 바라보며 커다란 숨을 쉬니 드높은 하늘처럼 내 마음 편해지네. 텅 빈 하늘 언제 왔나 고추잠자리 하나가 잠 덜 깬 듯 엉성히 돌기만 비잉비잉 음 토닥토닥 빨래하는 어머니의 분주함과 동기동기 기타 치는 그 아들의 한가함이 심심하면 쳐대는 괘종시계 종소리와 시끄러운 조카들의 울음소리 어우러진 가을 아침 내겐 정말 커다란 기쁨이야. 가을 아침 내겐 정말 커다란 행복이야. 응석만 부렸던 내겐 가을 아침 내겐 정말 커다란 기쁨이야. 가을 아침 내겐 정말 커다란 행복이야. 뜬구름 쫓았던 내겐 이른 아침 작은 새들 노랫소리 들려오면 언제나 그랬듯 아쉽게 잠을 깬다. 창문 하나 햇살 가득 눈부시게 비쳐오고 서늘한 냉기에 재채기할까 말까 음',
 '이른 아침 작은 새들 노랫소리 들려오면 언제나 그랬듯 아쉽게 잠을 깬다. 창문 하나 햇살 가득 눈부시게 비쳐오고 서늘한 냉기에 재채기할까 말까 음 눈 비비며 빼꼼히 창밖을 내다보니 삼삼오오 아이들은 재잘대며 학교 가고 산책 갔다 오시는 아버지의 양손에는 효과를 알 수 없는 약수가 하나 가득 음 딸각딸각 아침 짓는 어머니의 분주함과 엉금엉금 냉수 찾는 그 아들의 게으름이 상큼하고 깨끗한 아침의 향기와 구수하게 밥 뜸드는 냄새가 어우러진 가을 아침 내겐 정말 커다란 기쁨이야. 가을 아침 내겐 정말 커다란 행복이야. 응석만 부렸던 내겐 파란 하늘 바라보며 커다란 숨을 쉬니 드높은 하늘처럼 내 마음

In [ ]:
# 불용어 처리
stop_words="아 책 소설 서술 대한 위해 못 준 겐 음 이제 층 이따 꼭 접 막 난 다가 이일 걸 자주 고 여간 대 도 채 뭐 테 쪽 간 바 터 리 거기 듯 위 다 는걸 아주 휴 아이구 아이쿠 아이고 니 건 곳 어 나 저희 따라 의해 을 를 에 의 가 으로 로 에게 뿐이다 의거하여 근거하여 입각하여 기준으로 예하면 예를 들면 예를 들자면 저 소인 소생 저희 지말고 하지마 하지마라 다른 물론 또한 그리고 비길수 없다 해서는 안된다 뿐 거 뿐만 아니라 만이 아니다 만은 아니다 막론하고 관계없이 그치지 않다 그러나 그런데 하지만 든간에 논하지 않다 따지지 않다 설사 비록 더라도 아니면 만 못하다 하는 편이 낫다 불문하고 향하여 향해서 향하다 쪽으로 틈타 이용하여 타다 오르다 제외하고 이 외에 이 밖에 하여야 비로소 한다면 몰라도 외에도 이곳 여기 부터 기점으로 따라서 할 생각이다 하려고하다 이리하여 그리하여 그렇게 함으로써 하지만 일때 할때 앞에서 중에서 보는데서 으로써 로써 로서 때 까지 해야한다 일것이다 반드시 할줄알다 할수있다 할수있어 임에 틀림없다 한다면 등 등등 제 겨우 단지 다만 할뿐 딩동 댕그 대해서 대하여 대하면 훨씬 얼마나 얼마만큼 얼마큼 남짓 여 얼마간 약간 다소 좀 조금 다수 몇 얼마 지만 하물며 또한 그러나 그렇지만 하지만 이외에도 대해 말하자면 뿐이다 다음에 반대로 반대로 말하자면 이와 반대로 바꾸어서 말하면 바꾸어서 한다면 만약 그렇지않으면 까악 툭 딱 삐걱거리다 보드득 비걱거리다 꽈당 응당 해야한다 에 가서 각 각각 여러분 각종 각자 제각기 하도록하다 와 과 그러므로 그래서 고로 한 까닭에 하기 때문에 거니와 이지만 대하여 관하여 관한 과연 실로 아니나다를가 생각한대로 진짜로 한적이있다 하곤하였다 하 하하 허허 아하 거바 와 오 왜 어째서 무엇때문에 어찌 하겠는가 무슨 어디 어느곳 더군다나 하물며 더욱이는 어느때 언제 야 이봐 어이 여보시오 흐흐 흥 휴 헉헉 헐떡헐떡 영차 여차 어기여차 끙끙 아야 앗 아야 콸콸 졸졸 좍좍 뚝뚝 주룩주룩 솨 우르르 그래도 또 그리고 바꾸어말하면 바꾸어말하자면 혹은 혹시 답다 및 그에 따르는 때가 되어 즉 지든지 설령 가령 하더라도 할지라도 일지라도 지든지 몇 거의 하마터면 인젠 이젠 된바에야 된이상 만큼 어찌됏든 그위에 게다가 점에서 보아 비추어 보아 고려하면 하게될것이다 일것이다 비교적 좀 보다더 비하면 시키다 하게하다 할만하다 의해서 연이서 이어서 잇따라 뒤따라 뒤이어 결국 의지하여 기대여 통하여 자마자 더욱더 불구하고 얼마든지 마음대로 주저하지 않고 곧 즉시 바로 당장 하자마자 밖에 안된다 하면된다 그래 그렇지 요컨대 다시 말하자면 바꿔 말하면 즉 구체적으로 말하자면 시작하여 시초에 이상 허 헉 허걱 바와같이 해도좋다 해도된다 게다가 더구나 하물며 와르르 팍 퍽 푹 픽 폭 펄렁 동안 이래 하고있었다 이었다 에서 로부터 까지 예하면 했어요 해요 더불어 마저 마저도 양자 모두 습니다 가까스로 하려고하다 즈음하여 다른 다른 방면으로 해봐요 습니까 했어요 말할것도 없고 무릎쓰고 개의치않고 하는것만 못하다 하는것이 낫다 매 매번 들 모 어느것 어느 로써 갖고말하자면 어디 어느쪽 어느것 어느해 어느 년도 라 해도 언젠가 어떤것 어느것 저기 저쪽 저것 그때 그럼 그러면 요만한걸 그래 그때 저것만큼 그저 이르기까지 할 줄 안다 할 힘이 있다 너 너희 당신 어찌 설마 차라리 할지언정 할지라도 할망정 할지언정 구토하다 게우다 토하다 메쓰겁다 옆사람 퉤 쳇 의거하여 근거하여 의해 따라 힘입어 그 다음 버금 두번째로 기타 첫번째로 나머지는 그중에서 견지에서 형식으로 쓰여 입장에서 위해서 단지 의해되다 하도록시키다 뿐만아니라 반대로 전후 전자 앞의것 잠시 잠깐 하면서 그렇지만 다음에 그러한즉 그런즉 남들 아무거나 어찌하든지 같다 비슷하다 예컨대 이럴정도로 어떻게 만약 만일 위에서 서술한바와같이 인 듯하다 하지 않는다면 만약에 무엇 무슨 어느 어떤 아래윗 조차 한데 그럼에도 불구하고 여전히 심지어 까지도 조차도 하지 않도록 않기 위하여 때 시각 무렵 시간 동안 어때 어떠한 하여금 네 예 우선 누구 누가 알겠는가 아무도 줄은모른다 줄은 몰랏다 하는 김에 겸사겸사 하는바 그런 까닭에 한 이유는 그러니 그러니까 때문에 그 너희 그들 너희들 타인 것 것들 너 위하여 공동으로 동시에 하기 위하여 어찌하여 무엇때문에 붕붕 윙윙 나 우리 엉엉 휘익 윙윙 오호 아하 어쨋든 만 못하다 하기보다는 차라리 하는 편이 낫다 흐흐 놀라다 상대적으로 말하자면 마치 아니라면 쉿 그렇지 않으면 그렇지 않다면 안 그러면 아니었다면 하든지 아니면 이라면 좋아 알았어 하는것도 그만이다 어쩔수 없다 하나 일 일반적으로 일단 한켠으로는 오자마자 이렇게되면 이와같다면 전부 한마디 한항목 근거로 하기에 아울러 하지 않도록 않기 위해서 이르기까지 이 되다 로 인하여 까닭으로 이유만으로 이로 인하여 그래서 이 때문에 그러므로 그런 까닭에 알 수 있다 결론을 낼 수 있다 으로 인하여 있다 어떤것 관계가 있다 관련이 있다 연관되다 어떤것들 에 대해 이리하여 그리하여 여부 하기보다는 하느니 하면 할수록 운운 이러이러하다 하구나 하도다 다시말하면 다음으로 에 있다 에 달려 있다 우리 우리들 오히려 하기는한데 어떻게 어떻해 어찌됏어 어때 어째서 본대로 자 이 이쪽 여기 이것 이번 이렇게말하자면 이런 이러한 이와 같은 요만큼 요만한 것 얼마 안 되는 것 이만큼 이 정도의 이렇게 많은 것 이와 같다 이때 이렇구나 것과 같이 끼익 삐걱 따위 와 같은 사람들 부류의 사람들 왜냐하면 중의하나 오직 오로지 에 한하다 하기만 하면 도착하다 까지 미치다 도달하다 정도에 이르다 할 지경이다 결과에 이르다 관해서는 여러분 하고 있다 한 후 혼자 자기 자기집 자신 우에 종합한것과같이 총적으로 보면 총적으로 말하면 총적으로 대로 하다 으로서 참 그만이다 할 따름이다 쿵 탕탕 쾅쾅 둥둥 봐 봐라 아이야 아니 와아 응 아이 참나 년 월 일 령 영 일 이 삼 사 오 육 륙 칠 팔 구 이천육 이천칠 이천팔 이천구 하나 둘 셋 넷 다섯 여섯 일곱 여덟 아홉 령 영 이 있 하 것 들 그 되 수 이 보 않 없 나 사람 주 아니 등 같 우리 때 년 가 한 지 대하 오 말 일 그렇 위하 때문 그것 두 말하 알 그러나 받 못하 일 그런 또 문제 더 사회 많 그리고 좋 크 따르 중 나오 가지 씨 시키 만들 지금 생각하 그러 속 하나 집 살 모르 적 월 데 자신 안 어떤 내 내 경우 명 생각 시간 그녀 다시 이런 앞 보이 번 나 다른 어떻 여자 개 전 들 사실 이렇 점 싶 말 정도 좀 원 잘 통하 놓"
stop_words=stop_words.split(' ')

In [ ]:
#puctuation 제거
from string import punctuation
def strip_punctuation(s):
    return ''.join(c for c in s if c not in punctuation)

sentence =[]
for sent in all_contents:
    clean_text = strip_punctuation(sent)
    sentence.append(clean_text)
    
sentence

['이른 아침 작은 새들 노랫소리 들려오면 언제나 그랬듯 아쉽게 잠을 깬다 창문 하나 햇살 가득 눈부시게 비쳐오고 서늘한 냉기에 재채기할까 말까 음 눈 비비며 빼꼼히 창밖을 내다보니 삼삼오오 아이들은 재잘대며 학교 가고 산책 갔다 오시는 아버지의 양손에는 효과를 알 수 없는 약수가 하나 가득 음 딸각딸각 아침 짓는 어머니의 분주함과 엉금엉금 냉수 찾는 그 아들의 게으름이 상큼하고 깨끗한 아침의 향기와 구수하게 밥 뜸드는 냄새가 어우러진 가을 아침 내겐 정말 커다란 기쁨이야 가을 아침 내겐 정말 커다란 행복이야 응석만 부렸던 내겐 파란 하늘 바라보며 커다란 숨을 쉬니 드높은 하늘처럼 내 마음 편해지네 텅 빈 하늘 언제 왔나 고추잠자리 하나가 잠 덜 깬 듯 엉성히 돌기만 비잉비잉 음 토닥토닥 빨래하는 어머니의 분주함과 동기동기 기타 치는 그 아들의 한가함이 심심하면 쳐대는 괘종시계 종소리와 시끄러운 조카들의 울음소리 어우러진 가을 아침 내겐 정말 커다란 기쁨이야 가을 아침 내겐 정말 커다란 행복이야 응석만 부렸던 내겐 가을 아침 내겐 정말 커다란 기쁨이야 가을 아침 내겐 정말 커다란 행복이야 뜬구름 쫓았던 내겐 이른 아침 작은 새들 노랫소리 들려오면 언제나 그랬듯 아쉽게 잠을 깬다 창문 하나 햇살 가득 눈부시게 비쳐오고 서늘한 냉기에 재채기할까 말까 음',
 '이른 아침 작은 새들 노랫소리 들려오면 언제나 그랬듯 아쉽게 잠을 깬다 창문 하나 햇살 가득 눈부시게 비쳐오고 서늘한 냉기에 재채기할까 말까 음 눈 비비며 빼꼼히 창밖을 내다보니 삼삼오오 아이들은 재잘대며 학교 가고 산책 갔다 오시는 아버지의 양손에는 효과를 알 수 없는 약수가 하나 가득 음 딸각딸각 아침 짓는 어머니의 분주함과 엉금엉금 냉수 찾는 그 아들의 게으름이 상큼하고 깨끗한 아침의 향기와 구수하게 밥 뜸드는 냄새가 어우러진 가을 아침 내겐 정말 커다란 기쁨이야 가을 아침 내겐 정말 커다란 행복이야 응석만 부렸던 내겐 파란 하늘 바라보며 커다란 숨을 쉬니 드높은 하늘처럼 내 마음 편해지네 텅 빈 하늘

In [ ]:
# def cleaning(text):
#     from string import punctuation
#     text=''.join(c for c in text if c not in punctuation)
#     pattern = '[^ ㄱ-ㅣ가-힣|0-9|a-zA-Z]+'
#     text=re.sub(pattern=pattern, repl='', string=text)
#     return text

In [ ]:
# 전체 데이터 POS tagging
# NNG 일반명사, NNP 고유명사 -> nouns 이것만 남기기
def pos_tagging(text):
    pos_tagged=[]
    for t in text:
        tagged=mecab.pos(t)
        pos_tagged.append(tagged)

    return pos_tagged

In [ ]:
result=[]
for i in pos_tagged:
    nouns = [n for n, t in i if t in ['NNG', 'NNP']]
    n=[]
    for w in nouns:
        if w not in stop_words:
            n.append(w)

In [ ]:
df

,filename,contents,nouns,keywords,topic
0,가을아침.txt,이른 아침 작은 새들 노랫소리 들려오면 언제나 그랬듯 아쉽게 잠을 깬다. 창문 하나...,NaN,NaN,NaN
1,Love Poem.txt,이른 아침 작은 새들 노랫소리 들려오면 언제나 그랬듯 아쉽게 잠을 깬다. 창문 하나...,NaN,NaN,NaN
2,밤편지.txt,이 밤 그날의 반딧불을 당신의 창 가까이 보낼게요. 음 사랑한다는 말이에요. 나 우...,NaN,NaN,NaN
3,아이와 나의 바다.txt,그러나 시간이 지나도 아물지 않는 일들이 있지 내가 날 온전히 사랑하지 못해서 맘이...,NaN,NaN,NaN
4,비밀의 화원.txt,바람을 타고 날아오르는 새들은 걱정 없이 아름다운 태양 속으로 음표가 되어 나네. ...,NaN,NaN,NaN
5,너의 의미.txt,너의 그 한 마디 말도 그 웃음도 나에겐 커다란 의미 너의 그 작은 눈빛도 쓸쓸한 ...,NaN,NaN,NaN
6,이름에게.txt,꿈에서도 그리운 목소리는 이름 불러도 대답을 하지 않아. 글썽이는 그 메아리만 돌아...,NaN,NaN,NaN
7,마음.txt,툭 웃음이 터지면 그건 너 쿵 내려앉으면은 그건 너 축 머금고 있다면 그건 너 둥 ...,NaN,NaN,NaN
8,무릎.txt,모두 잠드는 밤에 혼자 우두커니 앉아. 다 지나버린 오늘을 보내지 못하고서 깨어있어...,NaN,NaN,NaN
9,이런 엔딩.txt,안녕 오랜만이야. 물음표 없이 참 너다운 목소리 정해진 규칙처럼 추운 문가에 늘 똑...,NaN,NaN,NaN


In [ ]:
def get_nouns(pos_list):

    result=[]
    for i in pos_list:
        nouns = [n for n, t in i if t in ['NNG', 'NNP']]
        for w in nouns:
            if w not in stop_words:
                result.append(w)
    return result

In [ ]:
# cleaned=cleaning(all_contents)
pos_tagged=pos_tagging(sentence)
pos_tagged

[[('이른', 'VA+ETM'),
  ('아침', 'NNG'),
  ('작', 'VA'),
  ('은', 'ETM'),
  ('새', 'NNG'),
  ('들', 'XSN'),
  ('노랫소리', 'NNG'),
  ('들려오', 'VV'),
  ('면', 'EC'),
  ('언제나', 'MAG'),
  ('그랬', 'VV+EP'),
  ('듯', 'EC'),
  ('아쉽', 'VA'),
  ('게', 'EC'),
  ('잠', 'NNG'),
  ('을', 'JKO'),
  ('깬다', 'VV+EC'),
  ('창문', 'NNG'),
  ('하나', 'NR'),
  ('햇살', 'NNG'),
  ('가득', 'MAG'),
  ('눈부시', 'VA'),
  ('게', 'EC'),
  ('비쳐', 'VV+EC'),
  ('오', 'VV'),
  ('고', 'EC'),
  ('서늘', 'XR'),
  ('한', 'XSA+ETM'),
  ('냉기', 'NNG'),
  ('에', 'JKB'),
  ('재채기', 'NNG'),
  ('할까', 'XSV+EC'),
  ('말', 'VX'),
  ('까', 'EF'),
  ('음', 'IC'),
  ('눈', 'NNG'),
  ('비비', 'VV'),
  ('며', 'EC'),
  ('빼꼼히', 'MAG'),
  ('창밖', 'NNG'),
  ('을', 'JKO'),
  ('내다보', 'VV'),
  ('니', 'EC'),
  ('삼', 'NR'),
  ('삼', 'NR'),
  ('오', 'NR'),
  ('오', 'NR'),
  ('아이', 'NNG'),
  ('들', 'XSN'),
  ('은', 'JX'),
  ('재잘대', 'VV'),
  ('며', 'EC'),
  ('학교', 'NNG'),
  ('가', 'VV'),
  ('고', 'EC'),
  ('산책', 'NNG'),
  ('갔', 'VV+EP'),
  ('다', 'EC'),
  ('오', 'VX'),
  ('시', 'EP'),
  ('는', 'ETM'),
  

In [ ]:
result=get_nouns(pos_tagged)
len(result)

1376

In [ ]:
get_nouns(pos_tagged)

['아침',
 '새',
 '노랫소리',
 '잠',
 '창문',
 '햇살',
 '냉기',
 '재채기',
 '눈',
 '창밖',
 '학교',
 '산책',
 '아버지',
 '양손',
 '효과',
 '약수',
 '아침',
 '어머니',
 '분주',
 '냉수',
 '아들',
 '게으름',
 '아침',
 '향기',
 '구수',
 '밥',
 '뜸',
 '냄새',
 '가을',
 '아침',
 '기쁨',
 '가을',
 '아침',
 '행복',
 '응석',
 '하늘',
 '숨',
 '하늘',
 '마음',
 '하늘',
 '고추잠자리',
 '잉비',
 '빨래',
 '어머니',
 '분주',
 '동기',
 '동기',
 '아들',
 '가함',
 '괘종시계',
 '종소리',
 '조카',
 '울음',
 '소리',
 '가을',
 '아침',
 '기쁨',
 '가을',
 '아침',
 '행복',
 '응석',
 '가을',
 '아침',
 '기쁨',
 '가을',
 '아침',
 '행복',
 '뜬구름',
 '아침',
 '새',
 '노랫소리',
 '잠',
 '창문',
 '햇살',
 '냉기',
 '재채기',
 '아침',
 '새',
 '노랫소리',
 '잠',
 '창문',
 '햇살',
 '냉기',
 '재채기',
 '눈',
 '창밖',
 '학교',
 '산책',
 '아버지',
 '양손',
 '효과',
 '약수',
 '아침',
 '어머니',
 '분주',
 '냉수',
 '아들',
 '게으름',
 '아침',
 '향기',
 '구수',
 '밥',
 '뜸',
 '냄새',
 '가을',
 '아침',
 '기쁨',
 '가을',
 '아침',
 '행복',
 '응석',
 '하늘',
 '숨',
 '하늘',
 '마음',
 '하늘',
 '고추잠자리',
 '잉비',
 '빨래',
 '어머니',
 '분주',
 '동기',
 '동기',
 '아들',
 '가함',
 '괘종시계',
 '종소리',
 '조카',
 '울음',
 '소리',
 '가을',
 '아침',
 '기쁨',
 '가을',
 '아침',
 '행복',
 '응석',
 '가을',
 '아침',
 '기쁨',
 '가을

In [ ]:
dataset = [h.split() for h in cleaned]
dictionary = corpora.Dictionary(dataset)
corpus=[dictionary.doc2bow(d) for d in dataset]

In [ ]:
dictionary

In [ ]:
corpus

[[(0, 1)],
 [(1, 1)],
 [],
 [(2, 1)],
 [(3, 1)],
 [],
 [(4, 1)],
 [(5, 1)],
 [],
 [(6, 1)],
 [(7, 1)],
 [],
 [(8, 1)],
 [(9, 1)],
 [(10, 1)],
 [(11, 1)],
 [],
 [(7, 1)],
 [(12, 1)],
 [(13, 1)],
 [(14, 1)],
 [],
 [(15, 1)],
 [(16, 1)],
 [(17, 1)],
 [],
 [(18, 1)],
 [(19, 1)],
 [(20, 1)],
 [],
 [(2, 1)],
 [(21, 1)],
 [(22, 1)],
 [],
 [(23, 1)],
 [(24, 1)],
 [],
 [(25, 1)],
 [(26, 1)],
 [],
 [(27, 1)],
 [(28, 1)],
 [],
 [(29, 1)],
 [(17, 1)],
 [],
 [(30, 1)],
 [(31, 1)],
 [],
 [(32, 1)],
 [(33, 1)],
 [],
 [(34, 1)],
 [(35, 1)],
 [(36, 1)],
 [(22, 1)],
 [],
 [(37, 1)],
 [(38, 1)],
 [(13, 1)],
 [(39, 1)],
 [],
 [(40, 1)],
 [(41, 1)],
 [(42, 1)],
 [],
 [(43, 1)],
 [(44, 1)],
 [(45, 1)],
 [],
 [(46, 1)],
 [(47, 1)],
 [(44, 1)],
 [(48, 1)],
 [(49, 1)],
 [],
 [(50, 1)],
 [(49, 1)],
 [],
 [(51, 1)],
 [],
 [(34, 1)],
 [],
 [(37, 1)],
 [(37, 1)],
 [(52, 1)],
 [],
 [(53, 1)],
 [(54, 1)],
 [(55, 1)],
 [],
 [(27, 1)],
 [(56, 1)],
 [(24, 1)],
 [],
 [(57, 1)],
 [(26, 1)],
 [(58, 1)],
 [(59, 1)],
 [],
 

In [ ]:
def compute_coherence_values(dictionary, corpus, texts, limit, start=2, step=3):
    coherence_values = []
    model_list = []
    for num_topics in range(start, limit, step):
        model = LdaModel(corpus=corpus, num_topics=num_topics, id2word=dictionary)
        model_list.append(model)
        coherencemodel = CoherenceModel(model=model, texts=result, dictionary=dictionary, coherence='c_v')
        coherence_values.append(coherencemodel.get_coherence())

    return model_list, coherence_values


# Can take a long time to run.
model_list, coherence_values = compute_coherence_values(dictionary=dictionary, corpus=corpus, texts=dataset, start=4, limit=21, step=2)

NameError: ignored